In [5]:
import pandas as pd
import datetime as dt
import random
import plotly.express as px
import plotly.io as pio

The first portion of this code takes water usage paramaeters from the environmental team and applies them to the correct time of day and day of the week.

In [6]:
weekend_percentage = 0.1
weekday_water_use = 3558.55 #litres
weekend_water_use = weekday_water_use * weekend_percentage #litres
weekday_greywater_demand = 7614 #litres
weekend_greywater_demand = weekday_greywater_demand * weekend_percentage #litres
area_of_roof = 2856 #meteres squared
volume_of_tank = 12500 #litres
green_roof_usage_coefficient = 0.25

In [7]:
weekday_water_use_dict = {
    0 : 0,
    1 : 0,
    2 : 0,
    3 : 0,
    4 : 0,
    5 : 0,
    6 : 2.04,
    7 : 4.08,
    8 : 6.13,
    9 : 9.18,
    10 : 12.24,
    11 : 7.15,
    12 : 10.2,
    13 : 13.26,
    14 : 10.2,
    15 : 6.13,
    16 : 7.15,
    17 : 8.17,
    18 : 4.08,
    19 : 2.04,
    20 : 0,
    21 : 0,
    22 : 0,
    23 : 0
}

weekend_water_use_dict = {
    0 : 0,
    1 : 0,
    2 : 0,
    3 : 0,
    4 : 0,
    5 : 0,
    6 : 2.04,
    7 : 4.08,
    8 : 6.13,
    9 : 9.18,
    10 : 12.24,
    11 : 7.15,
    12 : 10.2,
    13 : 13.26,
    14 : 10.2,
    15 : 6.13,
    16 : 7.15,
    17 : 8.17,
    18 : 4.08,
    19 : 2.04,
    20 : 0,
    21 : 0,
    22 : 0,
    23 : 0
}

weekday_greywater_demand_dict = {
    0 : 100/9,
    1 : 100/9,
    2 : 100/9,
    3 : 100/9,
    4 : 100/9,
    5 : 100/9,
    6 : 0,
    7 : 0,
    8 : 0,
    9 : 0,
    10 : 0,
    11 : 0,
    12 : 0,
    13 : 0,
    14 : 0,
    15 : 0,
    16 : 0,
    17 : 0,
    18 : 0,
    19 : 0,
    20 : 0,
    21 : 100/9,
    22 : 100/9,
    23 : 100/9
}

weekend_greywater_demand_dict = {
    0 : 100/9,
    1 : 100/9,
    2 : 100/9,
    3 : 100/9,
    4 : 100/9,
    5 : 100/9,
    6 : 0,
    7 : 0,
    8 : 0,
    9 : 0,
    10 : 0,
    11 : 0,
    12 : 0,
    13 : 0,
    14 : 0,
    15 : 0,
    16 : 0,
    17 : 0,
    18 : 0,
    19 : 0,
    20 : 0,
    21 : 100/9,
    22 : 100/9,
    23 : 100/9
}

storm_dict = {
    0 : 3.55/100,
    1 : 15.39/100,
    2 : 54.44/100,
    3 : 22.48/100,
    4 : 4.14/100
}

In [8]:
#file_path = "C:/Users/ellis/OneDrive - Newcastle University/SESDP/Final Rainfall Datasets/DRIERtimeseries.csv" #Windows file path
file_path = "/Users/ellissk/Library/CloudStorage/OneDrive-NewcastleUniversity/SESDP/Final Rainfall Datasets/WETTERtimeseries.csv" #Macos file path
rainfall_data = pd.read_csv(file_path, index_col='Date')
rainfall_data.rename(columns={"Amount": "Rainfall Amount"}, inplace=True)

This section of code resamples the daily rainfall dataset, into an hourly dataset. Rainfall is distributed acording to a beta function over 5 hour storm durations at different points on each day there is rainfall.

In [9]:
#Create new model dataframe with hourly index values and rainfall distributed in 5 hour windows on a given day
def convert_daily_to_hourly(rainfall_data, storm_dict):
    if not isinstance(rainfall_data.index, pd.DatetimeIndex):
        rainfall_data.index = pd.to_datetime(rainfall_data.index, format="%d/%m/%Y")
    percentages = [storm_dict[k] for k in sorted(storm_dict)]
    window_length = len(percentages)
    hourly_rows = []
    for date, row in rainfall_data.iterrows():
        daily_value = row["Rainfall Amount"]
        day_hours = pd.date_range(start=date, periods=24, freq='h')
        rainfall_hourly = [0] * 24
        start_hour = random.randint(0, 24 - window_length)
        for offset, percent in enumerate(percentages):
            rainfall_hourly[start_hour + offset] = daily_value * percent
        for dt, rain in zip(day_hours, rainfall_hourly):
            hourly_rows.append((dt, rain))
    model = pd.DataFrame(hourly_rows, columns=["Date", "Rainfall Amount"])
    model.set_index("Date", inplace=True)
    return model

model = convert_daily_to_hourly(rainfall_data, storm_dict)

In [10]:
for current_time in model.index:
    if current_time.month in [6,7,8]:
        model.at[current_time, "Rainwater Harvested"] = 0
    else:
        model.at[current_time, "Rainwater Harvested"] = (model.at[current_time, "Rainfall Amount"] * area_of_roof) * (1 - green_roof_usage_coefficient)

In [11]:
#Define recycled water column in model
def recycled_water_function(value):
    if value.weekday() in [0, 1, 2, 3, 4]:
        return weekday_water_use * (weekday_water_use_dict[int(value.hour)] / 100)
    else:
        return weekend_water_use * (weekend_water_use_dict[int(value.hour)] / 100)
    
model["Recycled Greywater"] = model.index.map(recycled_water_function)

In [12]:
#Define greywater demand column in model
def greywater_demand_function(value):
    if value.weekday() in [0, 1, 2, 3, 4]:
        return weekday_greywater_demand * (weekday_greywater_demand_dict[int(value.hour)] / 100)
    else:
        return weekend_greywater_demand * (weekend_greywater_demand_dict[int(value.hour)] / 100)
    

model["Greywater Demand"] = model.index.map(greywater_demand_function)

The main function of the code calculates the volume of the tank, inflows and outflows to model the tank performance.

In [13]:
def storage_model(model,volume_of_tank):  
    model["Tank Volume"] = 0
    model["Tank Overflow"] = 0
    model["Tank Potable Demand"] = 0

    for current_time in model.index:
        prev_time = current_time - pd.Timedelta(hours=1)
        try:
            prev_tank_volume = model.at[prev_time, "Tank Volume"]
        except KeyError:
            prev_tank_volume = 0
        
        free_volume_tank = volume_of_tank - prev_tank_volume

        rainwater_harvested = model.at[current_time, "Rainwater Harvested"]
        recycled_greywater = model.at[current_time, "Recycled Greywater"]
        greywater_demand = model.at[current_time, "Greywater Demand"]

        tank_input = rainwater_harvested + recycled_greywater
        tank_output = greywater_demand

        if (free_volume_tank >= tank_input - tank_output) and (prev_tank_volume + tank_input - tank_output >= 0):
            new_tank_volume = prev_tank_volume + tank_input - tank_output
            tank_overflow = 0
            tank_potable_demand = 0
        elif (prev_tank_volume + tank_input - tank_output < 0):
            new_tank_volume = 0
            tank_potable_demand = -(prev_tank_volume + tank_input - tank_output)
            tank_overflow = 0
        else:
            new_tank_volume = volume_of_tank
            tank_overflow = tank_input - tank_output - free_volume_tank
            tank_potable_demand = 0
        
        if current_time.day == 1 and current_time.month in [3, 9]:
            new_tank_volume = 0

        model.at[current_time,"Tank Volume"] = new_tank_volume
        model.at[current_time, "Tank Overflow"] = tank_overflow
        model.at[current_time, "Tank Potable Demand"] = tank_potable_demand

storage_model(model, volume_of_tank)

/var/folders/py/p0bfkh6s563_2sk82v7g5wwr0000gn/T/ipykernel_37046/1826533633.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '72.59442000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  model.at[current_time,"Tank Volume"] = new_tank_volume
/var/folders/py/p0bfkh6s563_2sk82v7g5wwr0000gn/T/ipykernel_37046/1826533633.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '455.4544867059999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  model.at[current_time, "Tank Potable Demand"] = tank_potable_demand
/var/folders/py/p0bfkh6s563_2sk82v7g5wwr0000gn/T/ipykernel_37046/1826533633.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '198.54276571535684' has d

In [14]:
model

,Rainfall Amount,Rainwater Harvested,Recycled Greywater,Greywater Demand,Tank Volume,Tank Overflow,Tank Potable Demand
Date,,,,,,,
2025-01-01 00:00:00,0.0,0.0,0.00000,846.0,0.000000,0.0,846.0
2025-01-01 01:00:00,0.0,0.0,0.00000,846.0,0.000000,0.0,846.0
2025-01-01 02:00:00,0.0,0.0,0.00000,846.0,0.000000,0.0,846.0
2025-01-01 03:00:00,0.0,0.0,0.00000,846.0,0.000000,0.0,846.0
2025-01-01 04:00:00,0.0,0.0,0.00000,846.0,0.000000,0.0,846.0
...,...,...,...,...,...,...,...
2099-12-31 19:00:00,0.0,0.0,72.59442,0.0,8547.985097,0.0,0.0
2099-12-31 20:00:00,0.0,0.0,0.00000,0.0,8547.985097,0.0,0.0
2099-12-31 21:00:00,0.0,0.0,0.00000,846.0,7701.985097,0.0,0.0


The final section of code below calculates some statistics from the model and plots them.

In [15]:
percentage_of_greywater_demand_met = 1 - (model["Tank Potable Demand"].sum() / model["Greywater Demand"].sum())
perectnage_of_rainwater_overflowed = (model["Tank Overflow"].sum()) / (model["Rainwater Harvested"].sum())
percenate_of_days_demand_met = (model.resample('D')['Tank Potable Demand'].sum() == 0).mean()
print(perectnage_of_rainwater_overflowed)


0.4745120250716133


In [16]:
statistic_headers = ["Statistic", "Score"]
statistic_df = pd.DataFrame(columns=statistic_headers)
statistic_df["Statistic"] = ["Volumetric", "Temporal"]
statistic_df.set_index("Statistic", inplace=True)
statistic_df.at["Volumetric", "Score"] = round(percentage_of_greywater_demand_met * 100,2)
statistic_df.at["Temporal", "Score"] = round(percenate_of_days_demand_met* 100,2)
statistic_df

,Score
Statistic,
Volumetric,81.14
Temporal,69.27


In [17]:
fig = px.bar(statistic_df, 
             x=statistic_df.index, 
             y='Score', 
             orientation='v',
             title='',
             labels={'Score': 'Percentage', 'Statistic': ''},
             text='Score',
             width=360,
             height=1400
             )

fig.update_traces(
    textposition='outside',
    marker_color='#7D9A79',              # Set the bar colour
    textfont=dict(color='#3E675D', size=32)  # Set text colour and increase text size
)

fig.update_xaxes(
    tickfont=dict(color='#3E675D', size=25)
)

fig.update_yaxes(
    range=[0, 100]
)

fig.update_layout(
    paper_bgcolor='#FFE7E1',  # Set the margin (paper) background colour
    plot_bgcolor='#FFE7E1'    # Set the plot background colour
)

fig.show()

In [18]:
start_date = "2050-01-01"
end_date = "2050-12-31"

In [19]:
df0 = rainfall_data
df0 = df0.loc[(df0.index >= start_date) & (df0.index <= end_date)]
fig0 = px.bar(
    df0,
    title="",
    labels={
        "value": "Rainfall Intensity (mm / hour)",
        "Date": "Date & Time"
    },
    width=2500,
    height=1400
)

fig0.update_traces(
    marker_color='#7D9A79',                # Set the bar colour
    textfont=dict(color='#3E675D', size=16)  # Set text colour and increase text size (if text is added)
)

fig0.update_layout(
    showlegend=False,
    plot_bgcolor='#FFFFFF',    # Set the plot background colour
    paper_bgcolor='#FFE7E1',   # Set the margin (paper) background colour
    font=dict(color="#3E675D", size=45),      # Set default text colour and increase text size
    title_font=dict(color="#3E675D", size=20)   # Set title text colour and size
)

fig0.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    tickformat="%B",
    tickfont=dict(color='#3E675D', size=36)
)

fig0.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    tickfont=dict(color='#3E675D', size=36)
)

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 1400,
    'width': 2500,
    'scale':6 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig0.show(config=config)



In [20]:
df1 = model["Tank Volume"]
df1 = df1.loc[(df1.index >= start_date) & (df1.index <= end_date)]
fig1 = px.line(
    df1,
    title="",
    labels={
        "value": "Tank Volume (litres)",
        "Date": "Date & Time"
    },
    width=2500,
    height=800
)

# Update trace to change the line colour to #7D9A79
fig1.update_traces(line=dict(color="#7D9A79"))

# Update layout for background, font and title text colour/size
fig1.update_layout(
    showlegend=False,
    plot_bgcolor="#FFFFFF",   # Set plot background colour #FFE7E1
    paper_bgcolor="#FFE7E1",  # Set margin (paper) background colour
    font=dict(color="#3E675D", size=45),       # Set default text colour and increase text size
    title_font=dict(color="#3E675D", size=20)    # Set title text colour and size
)

# Update x-axis styling: display only months and enlarge the x-axis text
fig1.update_xaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
    tickformat="%B",   # Display full month names (use "%b" for abbreviated names)
    tickfont=dict(color="#3E675D", size=35)  # Increase the size of the x-axis text
)

# Update y-axis styling and text properties
fig1.update_yaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
    range=[0, volume_of_tank],
    tickfont=dict(color="#3E675D", size=35)
)

fig1.show()



In [21]:
df2 = model["Tank Potable Demand"]
df2 = df2.loc[(df2.index >= start_date) & (df2.index <= end_date)]
fig2 = px.line(
    df2,
    title="",
    labels={
        "value": "Tank Demand (litres / hour)",
        "Date": "Date & Time"
    },
    width=2500,
    height=800
)

# Update trace to change the line colour to #7D9A79
fig2.update_traces(line=dict(color="#7D9A79"))

# Update layout for background, font and title text colour/size
fig2.update_layout(
    showlegend=False,
    plot_bgcolor="#FFFFFF",   # Set plot background colour
    paper_bgcolor="#FFE7E1",  # Set margin (paper) background colour
    font=dict(color="#3E675D", size=42),       # Set default text colour and increase text size
    title_font=dict(color="#3E675D", size=20)    # Set title text colour and size
)

# Update x-axis styling and text properties
fig2.update_xaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
    tickformat="%B",
    tickfont=dict(color="#3E675D", size=35)
)

# Update y-axis styling and text properties
fig2.update_yaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
    range=[0, 800],
    tickfont=dict(color="#3E675D", size=30)
)

fig2.show()
